In [ ]:
import os
import pandas as pd
from utils.voc2json import voc2json
from utils.mmdetection2wbf import *
from utils.dicom2png import dicom2png
from utils.preprocess import custom_Dataset

## Convert Dıcom files to PNG format

In [ ]:
dicom2png=dicom2png(dicom_path='./dıcom', png_path='./png')

In [ ]:
dicom2png.run()

##  Run preprocessing 

In [ ]:
pre_process=custom_Dataset(source_dir="./source/", target_dir="./source/", resize_dim=800)

In [ ]:
pre_process.runPreProcess()

##  Convert XML labels to Json format for MMdetection usage

In [ ]:
xml2json=voc2json("./new_test/labels",'new_test')

In [ ]:
xml2json.run()

##  MMdetection Usage

In [ ]:
#python tools/train.py sabl_faster_rcnn_r50_fpn_1x_coco.py

In [ ]:
#python tools/test.py sabl_faster_rcnn_r50_fpn_1x_coco.py sabl_faster_rcnn_r50_fpn_1x_coco/epoch_12.pth --format-only --options "jsonfile_prefix=./json/sabl_faster_rcnn_r50_fpn_1x_coco"

## Convert JSON outputs that output of evaluation, to CSV for WBF usage.

In [ ]:
mmdet2wbf=mmdetection2wbf(gt_path="./ground_truth/new_test.json", json_path="./json",csv_path="./csv", init=True)

In [ ]:
mmdet2wbf.main()

## Convert CSV to txt format for Metrics usage

In [ ]:
def get_Folder():
    for file in os.listdir("./csv/"):
        if file.endswith(".csv"):
            if os.path.exists(os.path.splitext(file)[0])==False:
                os.mkdir(os.path.splitext(file)[0])
                with open(file, 'r') as _file:
                    _df = pd.read_csv(_file)
                os.chdir(str(os.path.splitext(file)[0]))
                create_TXT(_df)
                os.chdir("../")
            else:
                print("File Already Exist In Same Directory")

In [ ]:
def create_TXT(df):
    for index, row in df.iterrows():
        img_id=row['img_id']
        _det_f_Name=str(int(img_id))+".txt"
        _det_F= open(_det_f_Name,"a+")
        class_id=str(int(row['label']))
        confidence=str(row['score'])
        x_center=str(row['x1']+(row['x2']-row['x1'])/2)
        y_center=str(row['y1']+(row['y2']-row['y1'])/2)
        width=str(row['x2']-row['x1'])
        height=str(row['y2']-row['y1'])
        det_ınfo=class_id+" "+confidence+" "+x_center+" "+y_center+" "+width+" "+height+"\n"
        _det_F.write(det_ınfo)
        _det_F.close()

In [ ]:
get_Folder()